### =================================================
# AI/ML Internship Assignment with Groq Api
## Tasks
### 1. Conversation Management and Summarization
### 2. JSON Schema classification and Extraction
### =================================================

In [3]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from openai import OpenAI
import requests
import json


## Setup API key

In [ ]:
os.environ["GROQ_API_KEY"]="YOUR_GROQ_API_KEY"


In [6]:
client=OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

### Task-1 Conversation Management with Summarization


In [7]:
class ConversationManager:
    def __init__(self, model="openai/gpt-oss-20b", k=3):
        self.model = model
        self.k = k
        self.history = []
        self.run_count = 0

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})
        self.run_count += 1

        # Summarize every k runs
        if self.run_count % self.k == 0:
            self.summarize()

    def summarize(self):
        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "Summarize this conversation briefly."},
                {"role": "user", "content": str(self.history)}
            ]
        )

        summary = response.choices[0].message.content
        # Replace history with summary
        self.history = [{"role": "system", "content": f"Summary: {summary}"}]

    def get_truncated_history(self, limit=3, by="turns"):
        # Truncate conversation by turns or characters
        if by == "turns":
            return self.history[-limit:]
        elif by == "chars":
            return [{"role": m["role"], "content": m["content"][:limit]} for m in self.history]
        else:
            return self.history


### Demo task 1

In [8]:
conv=ConversationManager(k=3)
# simulate conversation
conv.add_message("user","Hi i want to learn AI.")
conv.add_message("Assistant","Sure! AI means Artificial Intelligence.")
conv.add_message("user","Can you tell me about Machine learning?")

# summarization should trigger here

print("===Conversation after 3 turns with summarization===")
print(conv.history)

conv.add_message("assiastant","Machine Learning is the subset of AI.")
conv.add_message("user","What is Deep Learning")

print(conv.history)

===Conversation after 3 turns with summarization===
[{'role': 'system', 'content': 'Summary: **Conversation summary:**\n\n1. The user says they want to learn about AI.  \n2. The assistant briefly explains that AI stands for Artificial Intelligence.  \n3. The user then asks for more information about machine learning.'}]
[{'role': 'system', 'content': 'Summary: **Conversation summary:**\n\n1. The user says they want to learn about AI.  \n2. The assistant briefly explains that AI stands for Artificial Intelligence.  \n3. The user then asks for more information about machine learning.'}, {'role': 'assiastant', 'content': 'Machine Learning is the subset of AI.'}, {'role': 'user', 'content': 'What is Deep Learning'}]


In [9]:
print("\n=== Truncated History (last 2 turns) ===")
print(conv.get_truncated_history(limit=2, by="turns"))


=== Truncated History (last 2 turns) ===
[{'role': 'assiastant', 'content': 'Machine Learning is the subset of AI.'}, {'role': 'user', 'content': 'What is Deep Learning'}]


In [10]:
print("\n=== Truncated by Characters (first 20 chars each) ===")
print(conv.get_truncated_history(limit=20, by="chars"))


=== Truncated by Characters (first 20 chars each) ===
[{'role': 'system', 'content': 'Summary: **Conversat'}, {'role': 'assiastant', 'content': 'Machine Learning is '}, {'role': 'user', 'content': 'What is Deep Learnin'}]


In [11]:
print("\n=== Truncated History (last 2 turns) ===")
print(conv.get_truncated_history(limit=2, by=""))


=== Truncated History (last 2 turns) ===
[{'role': 'system', 'content': 'Summary: **Conversation summary:**\n\n1. The user says they want to learn about AI.  \n2. The assistant briefly explains that AI stands for Artificial Intelligence.  \n3. The user then asks for more information about machine learning.'}, {'role': 'assiastant', 'content': 'Machine Learning is the subset of AI.'}, {'role': 'user', 'content': 'What is Deep Learning'}]


### Task 2...

In [12]:
user_schema = {
    "name":"Extract_user_info",
    "description":"Extract user details from a chat",
    "parameters":{
        "type":"object",
        "properties":{
            "name":{
                "type":"string",
                "description":"The name of the user"
            },
            "email":{
                "type":"string",
                "description":"The email of the user"
            },
            "phone":{
                "type":"string",
                "description":"The phone number of the user"
            },
            "location":{
                "type":"string",
                "description":"The location of the user"
            },
            "age":{
                "type":"integer",
                "description":"The age of the user"
            }
        },
        "required":["name","email","phone","location","age"]
    }
}

In [13]:
def extract_info(chat_text):
  response = client.chat.completions.create(
      model="openai/gpt-oss-20b",
      messages=[
          {"role":"system","content":"Extract user details into JSON"},
          {"role":"user","content":chat_text}
      ],
      functions=[user_schema],
      function_call={"name":"Extract_user_info"}
  )
  return json.loads(response.choices[0].message.function_call.arguments)

### Demo

In [14]:
samples = [
    "Hi, I'm Roopendra and i am 21 year old currently working in Bangalore and my mobile number is 8519804772 an my mail id is mardalaroopendra@gmail.com",
    "Hello, this is kavitha from Delhi. I'm 30 years old. You can reach me at kavita@gmail.com or 9123456789.",
    "My name is charan, 27 years old, based in Mumbai.  9988776655."
]

print("====JSON Extraction Results ====")
for s in samples:
  print(extract_info(s))

====JSON Extraction Results ====
{'age': 21, 'email': 'mardalaroopendra@gmail.com', 'location': 'Bangalore', 'name': 'Roopendra', 'phone': '8519804772'}
{'age': 30, 'email': 'kavita@gmail.com', 'location': 'Delhi', 'name': 'Kavitha', 'phone': '9123456789'}
{'age': 27, 'email': '', 'location': 'Mumbai', 'name': 'charan', 'phone': '9988776655'}
